<a href="https://colab.research.google.com/github/sina1998/block-explorer-tutorials/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier

from collections import defaultdict

from nltk.probability import FreqDist, ELEProbDist

import nltk.classify.util  # for accuracy 


In [4]:
df_train_text = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment analysis/Train-Data.csv')

In [ ]:
df_train_text

,ID,text
0,b6f09f87d0a94e928722319748ec87af,I grew up (b. 1965) watching and loving the Th...
1,c477e2fafcd047d4a3b8cc4a86a8b5f4,"When I put this movie in my DVD player, and sa..."
2,e37706ad06d44b3a88525f3efec1be5f,Why do people who do not know what a particula...
3,e2ccf6bec7b44a869734ba9aa6700a43,Even though I have great interest in Biblical ...
4,03769b564ad442e4ac472a0c931d130d,Im a die hard Dads Army fan and nothing will e...
...,...,...
39995,414eb5b860974d14a0cc254e66c12afd,"""Western Union"" is something of a forgotten cl..."
39996,af30176598004044acf19c2064c3e6e2,This movie is an incredible piece of work. It ...
39997,d4936728110942ab926e184de8ded7e3,My wife and I watched this movie because we pl...
39998,90379ee3dee64afdabb6607d83f7b08e,"When I first watched Flatliners, I was amazed...."


In [5]:
df_train_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment analysis/Train-GT.csv')

In [6]:
df_train_label

,ID,label
0,b6f09f87d0a94e928722319748ec87af,0
1,c477e2fafcd047d4a3b8cc4a86a8b5f4,0
2,e37706ad06d44b3a88525f3efec1be5f,0
3,e2ccf6bec7b44a869734ba9aa6700a43,0
4,03769b564ad442e4ac472a0c931d130d,1
...,...,...
39995,414eb5b860974d14a0cc254e66c12afd,1
39996,af30176598004044acf19c2064c3e6e2,1
39997,d4936728110942ab926e184de8ded7e3,0
39998,90379ee3dee64afdabb6607d83f7b08e,1


In [7]:
df_train_merged = (df_train_text.merge(df_train_label, left_on='ID', right_on='ID'))

In [8]:
df_train_merged

,ID,text,label
0,b6f09f87d0a94e928722319748ec87af,I grew up (b. 1965) watching and loving the Th...,0
1,c477e2fafcd047d4a3b8cc4a86a8b5f4,"When I put this movie in my DVD player, and sa...",0
2,e37706ad06d44b3a88525f3efec1be5f,Why do people who do not know what a particula...,0
3,e2ccf6bec7b44a869734ba9aa6700a43,Even though I have great interest in Biblical ...,0
4,03769b564ad442e4ac472a0c931d130d,Im a die hard Dads Army fan and nothing will e...,1
...,...,...,...
39995,414eb5b860974d14a0cc254e66c12afd,"""Western Union"" is something of a forgotten cl...",1
39996,af30176598004044acf19c2064c3e6e2,This movie is an incredible piece of work. It ...,1
39997,d4936728110942ab926e184de8ded7e3,My wife and I watched this movie because we pl...,0
39998,90379ee3dee64afdabb6607d83f7b08e,"When I first watched Flatliners, I was amazed....",1


In [9]:
# This is how the Naive Bayes classifier expects the input
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
df = df_train_merged.head(4000)

In [43]:
neg_reviews = []
pos_reviews = []
for index, row in df_train_merged.iterrows():
    words = row['text']
    words = word_tokenize(words)
    if row['label'] == 0:
        neg_reviews.append((create_word_features(words), "negative"))
    else:
        pos_reviews.append((create_word_features(words), "positive"))

In [45]:
len(pos_reviews)

19981

In [47]:
train_neg_reviews = neg_reviews
train_pos_reviews = pos_reviews

In [54]:
df_test_text = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment analysis/Test-Data.csv')
df_test_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment analysis/Test-GT.csv')
df_test_merged = (df_test_text.merge(df_test_label, left_on='ID', right_on='ID'))
df_test_merged

,ID,text,label
0,e52ef7b35e4a4dd0a100a6f390fda273,I always wrote this series off as being a comp...,0
1,0362b11f85ae46e198a21e2369c9688b,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,e99c9e75f993486590065d62cc03a351,This movie was so poorly written and directed ...,0
3,175e727e358b4045b866689d91212022,The most interesting thing about Miryang (Secr...,1
4,f6a33ac2ef824ce98f2b3e5cc31e6621,"when i first read about ""berlin am meer"" i did...",0
...,...,...,...
4995,cda627b52502448e95b7057e8961edaf,This is the kind of picture John Lassiter woul...,1
4996,65165af9543b4ceb9dad2eb8b6cfb679,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,f8ba30de137e414db8c8537197b161a1,NBC should be ashamed. I wouldn't allow my chi...,0
4998,1cdca3781e37456b8d478c7b1133553c,This movie is a clumsy mishmash of various gho...,0


In [52]:
test_neg_reviews = []
test_pos_reviews = []
for index, row in df_test_merged.iterrows():
    words = row['text']
    words = word_tokenize(words)
    if row['label'] == 0:
        test_neg_reviews.append((create_word_features(words), "negative"))
    else:
        test_pos_reviews.append((create_word_features(words), "positive"))

In [55]:
len(test_pos_reviews)

2541

In [56]:
train_set = train_neg_reviews + train_pos_reviews
test_set = test_neg_reviews + test_pos_reviews

In [59]:
classifier = NaiveBayesClassifier.train(train_set)

In [60]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)

86.86


In [61]:
review_santa = '''
 
It would be impossible to sum up all the stuff that sucks about this film, so I'll break it down into what I remember most strongly: a man in an ingeniously fake-looking polar bear costume (funnier than the "bear" from Hercules in New York); an extra with the most unnatural laugh you're ever likely to hear; an ex-dope addict martian with tics; kid actors who make sure every syllable of their lines are slowly and caaarreee-fulll-yyy prrooo-noun-ceeed; a newspaper headline stating that Santa's been "kidnaped", and a giant robot. Yes, you read that right. A giant robot.
 
The worst acting job in here must be when Mother Claus and her elves have been "frozen" by the "Martians'" weapons. Could they be *more* trembling? I know this was the sixties and everyone was doped up, but still.
'''
words = word_tokenize(review_santa)
words = create_word_features(words)
# print(words)
print(classifier.classify(words))

negative


In [62]:
review_spirit = '''
Spirited Away' is the first Miyazaki I have seen, but from this stupendous film I can tell he is a master storyteller. A hallmark of a good storyteller is making the audience empathise or pull them into the shoes of the central character. Miyazaki does this brilliantly in 'Spirited Away'. During the first fifteen minutes we have no idea what is going on. Neither does the main character Chihiro. We discover the world as Chihiro does and it's truly amazing to watch. But Miyazaki doesn't seem to treat this world as something amazing. The world is filmed just like our workaday world would. The inhabitants of the world go about their daily business as usual as full with apathy as us normal folks. Places and buildings are not greeted by towering establishing shots and majestic music. The fact that this place is amazing doesn't seem to concern Miyazaki.
 
What do however, are the characters. Miyazaki lingers upon the characters as if they were actors. He infixes his animated actors with such subtleties that I have never seen, even from animation giants Pixar. Twenty minutes into this film and I completely forgot these were animated characters; I started to care for them like they were living and breathing. Miyazaki treats the modest achievements of Chihiro with unashamed bombast. The uplifting scene where she cleanses the River God is accompanied by stirring music and is as exciting as watching gladiatorial combatants fight. Of course, by giving the audience developed characters to care about, the action and conflicts will always be more exciting, terrifying and uplifting than normal, generic action scenes. 
'''
# print(review_spirit)

words = word_tokenize(review_spirit, language="english")
words = create_word_features(words)
print(classifier.classify(words))

positive
